#### Average line length

In [1]:
import statistics

def analyze_code(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()
    line_lengths = [len(line) for line in lines]
    avg_line_length = sum(line_lengths) / len(line_lengths)
    line_length_variability = statistics.stdev(line_lengths)
    print("Average line length:", avg_line_length)
    print("Line length variability:", line_length_variability)

analyze_code("/Users/adrian/Desktop/cs-projects/ureca-code-readability/DatasetBW/snippets/1.jsnp")


Average line length: 20.4
Line length variability: 17.66477977344876


In [5]:
def comment_percentage(file_path,language):
    with open(file_path) as f:
        code_snippet = f.read()
    comment_count = 0
    total_count = 0
    in_comment = False
    if language in ['python', 'c', 'c++', 'java']:
        if language == 'python':
            comment_symbol = '#'
        elif language == 'c':
            comment_symbol = '//'
        elif language == 'c++':
            comment_symbol = '//'
        elif language == 'java':
            comment_symbol = '//'
        for line in code_snippet.split('\n'):
            total_count += 1
            if in_comment:
                if '*/' in line:
                    in_comment = False
                else:
                    comment_count += 1
                    continue
            if comment_symbol in line:
                comment_count += 1
            elif '/*' in line:
                in_comment = True
                comment_count += 1
        return (comment_count / total_count) * 100
    else:
        return "Invalid language"

comment_percentage("/Users/adrian/Desktop/cs-projects/ureca-code-readability/DatasetBW/snippets/7.jsnp", 'java')

41.66666666666667

In [ ]:
6